### Library

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [2]:
r = requests.get("https://www.yelp.com/biz/kia-of-marin-novato")
soup = BeautifulSoup(r.text, "html.parser")

In [3]:
r.status_code

200

In [4]:
comments =  []
userName = []
rating = []

- Yorumlar

In [5]:
soup_lis = soup \
 .find('ul', {'class': 'list__09f24__ynIEd'})\
 .find_next('li', {'class': 'y-css-1jp2syp'})\
 
for lis in soup_lis:
    li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
        .find_next("div", {'class': 'y-css-cluvhg'})\
        .find_all_next("p", {'class': 'comment__09f24__D0cxf y-css-h9c2fl'})
        
    for l in li:
        comments.append(l.text)

- Kullanıcı Adı

In [6]:
soup_lis = soup \
 .find('ul', {'class': 'list__09f24__ynIEd'})\
 .find_next('li', {'class': 'y-css-1jp2syp'})\
 
for lis in soup_lis:
    li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
        .find_next("span", {'class': 'y-css-w3ea6v'})\
        .find_all_next("a", {"class": "y-css-12ly5yx"})
        
    for l in li:
        if l.text != "1 photo":
            userName.append(l.text)



- Verdikleri Oylar

In [7]:
soup_lis = soup \
 .find('ul', {'class': 'list__09f24__ynIEd'})\
 .find_next('li', {'class': 'y-css-1jp2syp'})\
 
for lis in soup_lis:
    li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
        .find_next("div", {'class': 'y-css-19pbem2'})\
        .find_next("span", {'class': 'y-css-pw0opj'})\
        .find_all_next("div", attrs={"class": 'y-css-9tnml4'})
        
        
    for l in li:
        #print(l['aria-label'])
        rating.append(l['aria-label'])
    
    rating = rating[:10]
        

In [8]:
data = pd.DataFrame(userName, columns=["UserName"])
data["Rating"] = pd.DataFrame(rating, columns=["Rating"])
data["Comments"] = pd.DataFrame(comments, columns=["Comments"])

In [9]:
data.head()

,UserName,Rating,Comments
0,Maureen P.,5 star rating,Even an EV needs service from time to time. Ki...
1,Isis L.,5 star rating,I wasn't looking to buy a car as I was content...
2,Patrick H.,5 star rating,Worked with Julio and he was excellent from st...
3,Koichi D.,5 star rating,Been dealing with Kia Marin for years. Leased ...
4,Janessa B.,5 star rating,Been bringing my Kia k5 here for a couple of y...
